# Identificando e Mapeando Zonas de Wifi

# Dependências

Esse projeto usa as bibliotecas `pandas` e `folium`. Elas serão instaladas automaticamente se não estiverem presentes.

In [31]:
import os
import re

os.system("pip install folium pandas &> /dev/null")

import pandas as pd

import folium
from folium.plugins import HeatMap

# Limpando Dados

Extraimos apenas os dados que nos interessam dos arquivos csv:
- `wifi.csv`.
- `gps.csv`.

## Limpando wifi.csv

Mantemos apenas as linhas que contêm o SSID "eduroam", uma vez que não nos interessamos em outros SSIDs, como "O que um macaco come?" (sim, esse é o nome de um SSID real).

Também removemos as colunas que não nos interessam, como a coluna de MAC address, que não é necessária para o nosso propósito.

Ademais, convertemos a coluna de tempo do formato datetime para o formato Unix timestamp, que é mais fácil de manipular e comparar. Também extraimos apenas os valores numéricos das colunas de `level` e `frequency`, pois os valores originais continham caracteres adicionais que não são necessários para a análise.

Finalmente, renomeamos as colunas para algo mais legível e removemos linhas duplicadas.

In [32]:
def clean_wifi_df(df: pd.DataFrame) -> pd.DataFrame:

	# Filter the data to only include rows where the SSID is "eduroam"
	df = df[df[1].str.contains("eduroam", na=False)]

	# Keep only colums 0, 2 and 5
	df = df[[0, 4, 5]]

	# Convert column '0' from YYYY-MM-DDTHH:MM:SS.sssZ to UNIX timestamp as integer representing milliseconds
	df[0] = pd.to_datetime(df[0]).astype(int) // 10 ** 6

	# Rename the columns
	df.columns = ["unix_ms", "level", "frequency"]

	# Convert all columns to integers
	df["level"] = [int(re.search(r"\-?\d+", x).group(0)) for x in df["level"]]
	df["frequency"] = [int(re.search(r"\d+", x).group(0)) for x in df["frequency"]]

	# Reset the index
	df.reset_index(drop=True, inplace=True)

	return df

def get_wifi(directory: str) -> pd.DataFrame:
	"""
	Reads the wifi.csv file from the given directory and returns a cleaned DataFrame.
	"""

	# Skip first line and ignore header, as original header is not useful.
	df = pd.read_csv(os.path.join(directory, "wifi.csv"), header=None, on_bad_lines='skip', skiprows=1)

	return clean_wifi_df(df)

## Limpando gps.csv

Similarmente à anterior, mantemos apenas as colunas que nos interessam, como `latitude`, `longitude` e `time`.

Novamente, convertemos a coluna de tempo do formato datetime para o formato Unix timestamp, que é mais fácil de manipular e comparar. Também convertemos as colunas de `latitude` e `longitude` para o tipo float, pois os valores originais estavam no formato string.

In [ ]:
def clean_gps_df(df: pd.DataFrame):

	df = df.copy()

	df = df[["datetime_utc", "latitude", "longitude"]]
	df.columns = ["unix_ms", "latitude", "longitude"]

	df["unix_ms"] = pd.to_datetime(df["unix_ms"]).astype(int) // 10 ** 6
	df["latitude"] = df["latitude"].astype(float)
	df["longitude"] = df["longitude"].astype(float)

	return df

def get_gps(directory: str) -> pd.DataFrame:
	"""
	Reads the gps.csv file from the given directory and returns a cleaned DataFrame.
	"""

	df = pd.read_csv(os.path.join(directory, "gps.csv"), on_bad_lines='skip')

	return clean_gps_df(df)

## Juntando os Dados

Juntamos os dados de `wifi.csv` e `gps.csv` com base no tempo, para que possamos analisar a intensidade do sinal de Wi-Fi em relação à localização GPS.

É importante notar que as leituras de Wi-Fi registram múltiplas frequências, como `2.4 GHz` e `5 GHz`, e cada uma delas pode ter um nível de sinal diferente. Portanto, para cada ponto GPS, podemos ter múltiplas leituras de Wi-Fi.

Por esse motivo nós agrupamos todas as leituras em dois grupos `2.4 GHz` e `5 GHz`, e calculamos a média dos níveis de sinal para cada grupo.

In [34]:
def join_dfs(wifi_df: pd.DataFrame, gps_df: pd.DataFrame):

	# Merge the two DataFrames on the 'unix_ms' column
	merged_df = pd.merge_asof(
		wifi_df.sort_values("unix_ms"),
		gps_df.sort_values("unix_ms"),
		on="unix_ms",
		direction="nearest",
		tolerance=1000  # tolerance in milliseconds
	)

	merged_df = merged_df[merged_df["latitude"].notna() & merged_df["longitude"].notna()]

	return merged_df

def average_dfs(dfs: list[pd.DataFrame]):
	"""
	Average the DataFrames in the list by 'unix_ms' and return a new DataFrame.
	"""

	if not dfs:
		return pd.DataFrame(columns=['unix_ms', 'level', 'latitude', 'longitude'])

	combined_df = pd.concat(dfs, ignore_index=True)

	averaged_df = combined_df.groupby('unix_ms', as_index=False)['level'].mean()
	first_occurrence = combined_df.drop_duplicates(subset='unix_ms')[['unix_ms', 'frequency', 'latitude', 'longitude']]
	result_df = pd.merge(averaged_df, first_occurrence, on='unix_ms')

	result_df.drop(columns=['frequency'], inplace=True)
	result_df.reset_index(drop=True, inplace=True)

	return result_df

def get_merged(path: str):

	wifi = pd.read_csv(os.path.join(path, "wifi.csv"), header=None, on_bad_lines='skip', skiprows=1)
	wifi = clean_wifi_df(wifi)

	gps = pd.read_csv(os.path.join(path, "gps.csv"), on_bad_lines='skip')
	gps = clean_gps_df(gps)

	joined = join_dfs(wifi, gps)

	wifi24 = []
	wifi5 = []

	for frequency in joined["frequency"].unique():

		value = frequency // 1000
		filtered = joined[joined["frequency"] == frequency]

		if value == 2:
			wifi24.append(filtered)
		elif value == 5:
			wifi5.append(filtered)

	return average_dfs(wifi24), average_dfs(wifi5)

# Visualizando os Dados

## Criando um Mapa de Calor

Para visualizar os dados, criamos um mapa de calor usando a biblioteca `folium` e `folium.plugins.HeatMap`.

Note que criamos dois mapas de calor, um para cada grupo de frequência (`2.4 GHz` e `5 GHz`), e salvamos os mapas em arquivos HTML separados.

Também abrimos os arquivos HTML no navegador para visualização.

In [35]:
"""
Espera-se a seguinte estrutura de pastas:
wifi-zones/
├── wifi-zones.ipynb
├── Data/
	├── rota1a - 2025-09-16-10-33-42-649/
		├── gps.csv
		├── wifi.csv
		...
		
	├── rota1a - 2025-09-16-10-33-42-649/
		...
	└── rota1a - 2025-09-16-10-33-42-649/
		...
	└── rota1a - 2025-09-16-10-33-42-649/
"""

DATA_FOLDER: str = "Data"
CSV_OUTPUT_FOLDER: str = "ProcessedData"
HTML_OUTPUT_FOLDER: str = "Heatmaps"

ROUTE_NAMES: list[str] = [d for d in os.listdir(DATA_FOLDER) if os.path.isdir(os.path.join(DATA_FOLDER, d))]


def plot_map(df: pd.DataFrame, output: str) -> None:
	
    df = df.copy()

    center_lat = df["latitude"].mean()
    center_lon = df["longitude"].mean()

    heat_data = [[row["latitude"], row["longitude"], row["level"]] for _, row in df.iterrows()]
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=18)

    HeatMap(heat_data, radius=15, blur=10, max_zoom=30).add_to(m)
    m.save(output)
	

def main() -> None:

    os.makedirs(CSV_OUTPUT_FOLDER, exist_ok=True)
    os.makedirs(HTML_OUTPUT_FOLDER, exist_ok=True)

    for route in ROUTE_NAMES:

        path = os.path.join(DATA_FOLDER, route)
        
        wifi24, wifi5 = get_merged(path)

        csv_path_24 = os.path.join(CSV_OUTPUT_FOLDER, f"wifi_2.4GHz_{route}.csv")
        html_path_24 = os.path.join(HTML_OUTPUT_FOLDER, f"heatmap_wifi_2.4GHz_{route}.html")
        
        wifi24.to_csv(csv_path_24, index=False)
        plot_map(wifi24, html_path_24)

        csv_path_5 = os.path.join(CSV_OUTPUT_FOLDER, f"wifi_5GHz_{route}.csv")
        html_path_5 = os.path.join(HTML_OUTPUT_FOLDER, f"heatmap_wifi_5GHz_{route}.html")

        wifi5.to_csv(csv_path_5, index=False)
        plot_map(wifi5, html_path_5)

if __name__ == "__main__":
	main()